If we want to save the results of analyzed high-resolution chunks of data in the storage, we need to provide a unique name for the file, which will contain information about the time grid or time band.

In [1]:
import sys
from aqua import Reader
sys.path.insert(0, '../../../')
from tropical_rainfall import Tropical_Rainfall

In [2]:
reader              = Reader(model="IFS", exp="tco2559-ng5-cycle3", source="lra-r100-monthly") 
ifs_tco2559_ng5     = reader.retrieve()
ifs_tco2559_ng5     = ifs_tco2559_ng5['tprate'][:]*86400 
ifs_tco2559_ng5.attrs['units'] = "mm/day"

2023-09-29 20:39:03 :: eccodes :: WARNING  -> ShortName tprate have multiple grib codes associated: ['260048', '172228']
2023-09-29 20:39:03 :: eccodes :: WARNING  -> AQUA will take the first so that tprate -> 260048, please set up a correct fix if this does not look right


In [3]:
diag = Tropical_Rainfall(trop_lat=15,  num_of_bins = 1000, first_edge = 0, width_of_bin = 1, loglevel='error')

Produced dataset (and dataarray) contains the attribute 'time_band.'

In [4]:
hist_ifs_tco2559_ng5 = diag.histogram(ifs_tco2559_ng5)
hist_ifs_tco2559_ng5.time_band

'2020-01-15T00:00:00.000000000, 2024-12-15T00:00:00.000000000, freq=M'

If we want to save the data in the storage, the name of the file will contain the time_band information.

In [5]:
diag.dataset_to_netcdf(hist_ifs_tco2559_ng5, path_to_netcdf='./', name_of_file='test')

2023-09-29 20:39:36 :: create_folder :: WARNING  -> Folder ./ already exists


'./trop_rainfall_test_2020-01-15T00_2024-12-15T00_M.nc'

In [6]:
ifs_tco2559_ng5_feb = ifs_tco2559_ng5.sel(time='2020-02')
hist_ifs_tco2559_ng5_feb = diag.histogram(ifs_tco2559_ng5_feb)
ifs_tco2559_ng5_mar = ifs_tco2559_ng5.sel(time='2020-03')
hist_ifs_tco2559_ng5_mar = diag.histogram(ifs_tco2559_ng5_mar)

If I want to merge two datasets (or even a list of datasets), the total time band will be appropriately updated. 

In [7]:
hist_ifs_tco2559_ng5_feb_mar  = diag.merge_two_datasets(hist_ifs_tco2559_ng5_feb, hist_ifs_tco2559_ng5_mar)
hist_ifs_tco2559_ng5_feb_mar.time_band

'2020-02-15T00:00:00.000000000, 2020-03-15T00:00:00.000000000, freq=1M'

The file's name in the filesystem will contain the correct total time band of the merged dataset. 

In [8]:
diag.dataset_to_netcdf(hist_ifs_tco2559_ng5_feb_mar, path_to_netcdf='./', name_of_file='test')

2023-09-29 20:40:41 :: create_folder :: WARNING  -> Folder ./ already exists


'./trop_rainfall_test_2020-02-15T00_2020-03-15T00_1M.nc'

Below is an example of how to merge the list of datasets stored in the filesystem. The produced dataset will contain updated 'time_band' and 'time_band_history.'

In [ ]:
path_to_ifs='/work/bb1153/b382267/tropical_rainfall_cicle3/NetCDF/histograms/tco2559-ng5-cycle3/'
hist_ifs_2559_high = diag.merge_list_of_histograms(path_to_histograms = path_to_ifs, multi = None, seasons = False, all = True)